In [ ]:
import os
import cv2
import glob
import random
import pydicom
import skimage.io
import numpy as np
import pandas as pd
from pydicom import dcmread
import matplotlib.pyplot as plt
from skimage.data import gravel
from scipy.fft import fftn, fftshift
from skimage.util import compare_images
from skimage.filters import difference_of_gaussians, window
from skimage.morphology import (erosion, dilation, opening, closing)

# Task 1 - Noise Removal

**Morphological**

*   erode
*   dilate
*   open
*   close


In [ ]:
inchi3 = cv2.imread('../input/picturefiles/inchi3.png')
speckle=cv2.imread("../input/picturefiles/3.png")
scooby=cv2.imread("../input/picturefiles/scooby_day.jpg")

In [ ]:
grayInchi3 = cv2.cvtColor(inchi3, cv2.COLOR_RGB2GRAY)
bwinchi3 = cv2.subtract(255, grayInchi3) 

In [ ]:
def display_output(img1,img2,filtertype):
    plt.figure(figsize=(11,6))
    plt.subplot(121),plt.imshow(img1,cmap='gray'),plt.title('Original')
    plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img2,cmap='gray'),plt.title(filtertype)
    plt.xticks([]), plt.yticks([])
    plt.show()

In [ ]:
kernel_size=(3,3)
kernel = np.ones(kernel_size, np.uint8)

closed = cv2.morphologyEx(bwinchi3, cv2.MORPH_CLOSE, kernel, iterations=1)
erosed = cv2.erode(closed, kernel_size, iterations=1)
dilated = cv2.dilate(erosed, kernel, iterations=1)
restoredmorph = cv2.subtract(255, dilated) 

In [ ]:
cv2.imwrite('final.png',restoredmorph)
display_output(inchi3,restoredmorph,'Morphological Filter')

**Spatial filters**

*   Mean 
*   Median
*   Gaussian 
*   Convolution
*   Bilateral 
*   Laplacian

In [ ]:
dilated = cv2.dilate(bwinchi3, kernel, iterations=1)
medianfilt= cv2.medianBlur(dilated, 5)
restoredmf = cv2.subtract(255, medianfilt) 
cv2.imwrite('medianfilt.png', restoredmf)
display_output(inchi3,restoredmf,'Median Filter')


In [ ]:
erosed = cv2.erode(bwinchi3, kernel_size, iterations=1)
gausfil = cv2.GaussianBlur(erosed, (3,3),0)
restoredgf = cv2.subtract(255, gausfil) 
cv2.imwrite('gaussianfilt.png', restoredgf)
display_output(inchi3,restoredgf,'Gaussian Filter')

In [ ]:
erosed = cv2.erode(bwinchi3, kernel_size, iterations=1)
kernel_conv = np.ones((3,3),np.float32)
conv_inchi3 = cv2.filter2D(erosed,0,kernel_conv)
restoredconv = cv2.subtract(255, conv_inchi3) 
cv2.imwrite('conv.png', restoredconv)
display_output(inchi3,restoredconv,'Convolution Filter')

**Frequency filter**

In [ ]:
windowIm = bwinchi3 * window('hann', bwinchi3.shape)  # window image to improve FFT
filteredIm = difference_of_gaussians(bwinchi3, 1, 12)
filteredIm_up = filteredIm * window('hann', bwinchi3.shape)
im_f_mag = fftshift(np.abs(fftn(windowIm)))
fim_f_mag = fftshift(np.abs(fftn(filteredIm_up)))

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
ax[0, 0].imshow(bwinchi3, cmap='gray')
ax[0, 0].set_title('Original Image')
ax[0, 1].imshow(np.log(im_f_mag), cmap='magma')
ax[0, 1].set_title('Original FFT Magnitude (log)')
ax[1, 0].imshow(filteredIm, cmap='gray')
ax[1, 0].set_title('Filtered Image')
ax[1, 1].imshow(np.log(fim_f_mag), cmap='magma')
ax[1, 1].set_title('Filtered FFT Magnitude (log)')
plt.show()

In [ ]:
windowIm = erosed * window('hann', erosed.shape)  # window image to improve FFT
filteredIm = difference_of_gaussians(erosed, 1.5)
filteredIm_up = filteredIm * window('hann', bwinchi3.shape)
im_f_mag = fftshift(np.abs(fftn(windowIm)))
fim_f_mag = fftshift(np.abs(fftn(filteredIm_up)))

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
ax[0, 0].imshow(erosed, cmap='gray')
ax[0, 0].set_title('Original Image')
ax[0, 1].imshow(np.log(im_f_mag), cmap='magma')
ax[0, 1].set_title('Original FFT Magnitude (log)')
ax[1, 0].imshow(filteredIm, cmap='gray')
ax[1, 0].set_title('Filtered Image')
ax[1, 1].imshow(np.log(fim_f_mag), cmap='magma')
ax[1, 1].set_title('Filtered FFT Magnitude (log)')
plt.show()

**Gaussian Noise Addition**


In [ ]:
scale_percent = 30 # percent of original size
width = int(scooby.shape[1] * scale_percent / 100)
height = int(scooby.shape[0] * scale_percent / 100)
dimension = (width, height)

# resize image
scooby_day_res = cv2.resize(scooby, dimension, interpolation = cv2.INTER_AREA)
img_noise=skimage.util.random_noise(scooby_day_res, mode='gaussian', seed=None, var=(15/255.0)**2) #add  gaussian noise
img_noise=img_noise*255 

In [ ]:
cv2.imwrite('gaussiannoise.jpg',img_noise)

In [ ]:
gausfil = cv2.GaussianBlur(img_noise, (9,9),5)

In [ ]:
cv2.imwrite('final_scoob.jpg',gausfil)

# Task 2 - Speckle Removal

In [ ]:
grayxray = cv2.cvtColor(speckle, cv2.COLOR_RGB2GRAY)
final_xray = cv2.fastNlMeansDenoising(grayxray)
cv2.imwrite('xray.jpg',final_xray)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage import io
from skimage.util import compare_images


def mean_filtering(input_image, window_size=(15, 15)):
    filtered_image = cv2.blur(input_image, window_size)
    filtered_image = np.nan_to_num(filtered_image)
    return filtered_image

def bilateral_filtering(input_image, window_size=15, sigma_color=5, sigma_space=5):
    filtered_image = cv2.bilateralFilter(input_image, window_size, sigma_color, sigma_space)
    filtered_image = np.nan_to_num(filtered_image)
    return filtered_image

def gaussian_filtering(input_image, window_size=(15, 15)):
    filtered_image = cv2.GaussianBlur(input_image, window_size, sigma_x, sigma_y)
    filtered_image = np.nan_to_num(filtered_image)
    return filtered_image

def exp_transformation(input_image):
    filtered_image = np.expm1(input_image)
    return filtered_image*(255/np.max(filtered_image))

def image_filters():

    image_ref = io.imread('../input/picturefiles/3.png', as_gray=True)

    fig, axs = plt.subplots(1, 3, figsize=(11, 6))
    axs[0].imshow(speckle, cmap='gray', vmin=0, vmax=255)
    axs[0].set_title("Original Image")

    log_input = np.log1p(image_ref).astype(np.float32)
    mean_output = exp_transformation(mean_filtering(log_input))
    axs[1].imshow(mean_output, cmap='gray')
    axs[1].set_title("Mean Filtering")

    gauss_mean = exp_transformation(bilateral_filtering(log_input))
    axs[2].imshow(gauss_mean, cmap='gray')
    axs[2].set_title("Gaussian Filtering")

    for j in range(3):
        axs[j].set_xticks([])
        axs[j].set_yticks([])
    plt.tight_layout()
    plt.show()
    cv2.imwrite('finalxray.jpg',gauss_mean)


image_filters()

# Task 3 - Visualization of the MRI data

In [ ]:
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/00015/"

In [ ]:
types=("FLAIR", "T1w", "T1wCE", "T2w")
fps = []
path = os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/00015/")
for i, t in enumerate(types, 1):
    t_paths = sorted(glob.glob(os.path.join(path, t, "*")))
    for file in t_paths:
        fps.append(file)

In [ ]:
dcm = dcmread(fps[200])

In [ ]:
print(dcm)

In [ ]:
plt.imshow(dcm.pixel_array, cmap=plt.cm.bone) 
plt.show()

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

def visualize_sample(slice_i, types=("FLAIR", "T1w", "T1wCE", "T2w")):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/00015/")
    for i, t in enumerate(types, 1):
        t_paths = sorted(glob.glob(os.path.join(patient_path, t, "*")), key=lambda x: int(x[:-4].split("-")[-1]))
        data = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")
    plt.show()


In [ ]:
visualize_sample(slice_i=0.5)